In [ ]:
#@title Könyvtárak telepítése
!pip install -U torch sentence-transformers accelerate pyarrow pandas tqdm transformers
print("Könyvtárak telepítve")


In [ ]:
#@title Qwen3-Embedding-8B Script - Optimalizált konfiguráció
import pandas as pd
import numpy as np
import gc
import json
import pyarrow as pa
import pyarrow.parquet as pq
from sentence_transformers import SentenceTransformer
import torch
import psutil
import time
import logging
from tqdm import tqdm
from typing import List, Dict, Any
import os
import warnings
warnings.filterwarnings('ignore')

# ============================================================
# OPTIMALIZÁLT KONFIGURÁLÁS - A100 80GB
# ============================================================

# Fájl elérési utak
INPUT_CSV = "cleaned_data_for_embedding.csv"
OUTPUT_PARQUET = "processed_documents_with_embeddings.parquet"
MODEL_NAME = "Qwen/Qwen3-Embedding-8B"

# A100 80GB optimalizált paraméterek
CHUNKSIZE = 10000          # Nagyobb chunk méret GPU-ra
BATCH_SIZE = 32            # Optimális batch méret A100-ra
MAX_TOKEN_LENGTH = 8192    # Qwen3-8B max context
PREFETCH_FACTOR = 4        # DataLoader prefetch
NUM_WORKERS = 8            # Párhuzamos adatbetöltés

# Fejlett GPU optimalizáció
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = False
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

# Mixed precision beállítások
USE_MIXED_PRECISION = True
AUTOCAST_ENABLED = True

# Memória optimalizáció
MEMORY_LIMIT_GB = 75       # A100 80GB-ból 75GB használata
GRADIENT_CHECKPOINTING = True

# Logging konfigurálás
logging.basicConfig(
    level=logging.INFO, 
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler('embedding_generation.log')
    ]
)
logger = logging.getLogger(__name__)

print("Optimalizált Qwen3-Embedding-8B Generátor")
print("=" * 60)
print(f"Modell: {MODEL_NAME}")
print(f"Dimenzió: 8192")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Chunk Size: {CHUNKSIZE:,}")
print(f"Memória limit: {MEMORY_LIMIT_GB}GB")
print(f"Mixed Precision: {USE_MIXED_PRECISION}")
print("=" * 60)


In [ ]:
#@title Segédfüggvények
def get_system_info():
    """Rendszerinformációk lekérdezése."""
    info = {
        'total_memory_gb': psutil.virtual_memory().total / (1024**3),
        'available_memory_gb': psutil.virtual_memory().available / (1024**3),
        'cpu_count': psutil.cpu_count(),
        'gpu_available': torch.cuda.is_available(),
        'gpu_count': torch.cuda.device_count() if torch.cuda.is_available() else 0
    }
    
    if torch.cuda.is_available():
        info['gpu_name'] = torch.cuda.get_device_name(0)
        info['gpu_memory_gb'] = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    
    return info

def adaptive_batch_size(text_lengths: List[int], base_batch_size: int = BATCH_SIZE) -> int:
    """Adaptív batch méret szöveg hossz alapján."""
    avg_length = np.mean(text_lengths)
    
    if avg_length > 6000:
        return max(8, base_batch_size // 4)
    elif avg_length > 4000:
        return max(16, base_batch_size // 2)
    elif avg_length > 2000:
        return base_batch_size
    else:
        return min(64, base_batch_size * 2)

def smart_memory_cleanup():
    """Intelligens memóriakezelés."""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
    gc.collect()
    
def estimate_processing_time(total_texts: int, batch_size: int, avg_time_per_batch: float) -> Dict[str, float]:
    """Becsült feldolgozási idő kalkuláció."""
    total_batches = (total_texts + batch_size - 1) // batch_size
    estimated_seconds = total_batches * avg_time_per_batch
    
    return {
        'total_batches': total_batches,
        'estimated_hours': estimated_seconds / 3600,
        'estimated_minutes': estimated_seconds / 60
    }

def create_metadata_json(row: pd.Series) -> str:
    """Optimalizált metadata JSON generálás."""
    metadata = {
        'doc_id': str(row.get('doc_id', '')),
        'birosag': str(row.get('birosag', '')),
        'jogterulet': str(row.get('jogterulet', '')),
        'hatarozat_id_mappa': str(row.get('hatarozat_id_mappa', '')),
        'text_length': len(str(row.get('text', ''))),
        'processed_timestamp': time.time()
    }
    return json.dumps(metadata, ensure_ascii=False)

# Rendszerinformációk megjelenítése
sys_info = get_system_info()
print("Rendszer információk:")
print("=" * 40)
for key, value in sys_info.items():
    print(f"{key}: {value}")
print("=" * 40)


In [ ]:
#@title Optimalizált Qwen3-8B Embedding Osztály
class OptimizedQwen3EmbeddingGenerator:
    """
    Optimalizált Qwen3-Embedding-8B implementáció A100 80GB GPU-ra.
    
    Főbb optimalizációk:
    - Mixed precision training (AMP)
    - Adaptív batch sizing
    - Memória monitoring és cleanup
    - Fejlett progress tracking
    - Chunk-based processing
    - Dynamic GPU memory management
    """
    
    def __init__(self):
        self.model_name = MODEL_NAME
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.dimension = 8192
        self.max_tokens = MAX_TOKEN_LENGTH
        
        # Performance tracking
        self.processed_count = 0
        self.failed_count = 0
        self.total_processing_time = 0
        self.batch_times = []
        
        # Memory monitoring
        self.peak_memory_usage = 0
        self.memory_warnings = 0
        
        logger.info(f"Optimalizált generátor inicializálása...")
        logger.info(f"Device: {self.device}")
        
        try:
            # Model loading with optimizations
            logger.info(f"Qwen3-Embedding-8B betöltése...")
            
            self.model = SentenceTransformer(
                self.model_name,
                device=self.device,
                trust_remote_code=True
            )
            
            # Model optimalizálás
            if self.device == 'cuda':
                self.model.half() if USE_MIXED_PRECISION else None
                
            # Warmup
            self._warmup_model()
            
            logger.info(f"Modell sikeresen betöltve és optimalizálva")
            
        except Exception as e:
            logger.error(f"Modell betöltési hiba: {e}")
            raise
    
    def _warmup_model(self):
        """Modell warmup a konzisztens teljesítményért."""
        logger.info("Modell warmup...")
        dummy_text = ["Ez egy teszt szöveg a modell bemelegítéséhez."] * 4
        
        with torch.cuda.amp.autocast(enabled=USE_MIXED_PRECISION and AUTOCAST_ENABLED):
            _ = self.model.encode(dummy_text, show_progress_bar=False)
        
        smart_memory_cleanup()
        logger.info("Warmup befejezve")
    
    def _monitor_memory(self) -> Dict[str, float]:
        """GPU memória monitoring."""
        if not torch.cuda.is_available():
            return {}
        
        allocated = torch.cuda.memory_allocated() / (1024**3)
        reserved = torch.cuda.memory_reserved() / (1024**3)
        max_allocated = torch.cuda.max_memory_allocated() / (1024**3)
        
        self.peak_memory_usage = max(self.peak_memory_usage, allocated)
        
        # Memory warning
        if allocated > MEMORY_LIMIT_GB * 0.9:
            self.memory_warnings += 1
            logger.warning(f"Magas memóriahasználat: {allocated:.1f}GB")
        
        return {
            'allocated_gb': allocated,
            'reserved_gb': reserved,
            'max_allocated_gb': max_allocated,
            'peak_usage_gb': self.peak_memory_usage
        }
    
    def _process_batch_with_optimization(self, batch_texts: List[str]) -> np.ndarray:
        """Optimalizált batch feldolgozás mixed precision-nel."""
        batch_start_time = time.time()
        
        try:
            # Szöveg előfeldolgozás
            processed_texts = []
            for text in batch_texts:
                if len(text) > self.max_tokens * 3:  # ~3 char/token
                    text = text[:self.max_tokens * 3]
                processed_texts.append(text)
            
            # Mixed precision embedding generálás
            with torch.cuda.amp.autocast(enabled=USE_MIXED_PRECISION and AUTOCAST_ENABLED):
                embeddings = self.model.encode(
                    processed_texts,
                    normalize_embeddings=True,
                    show_progress_bar=False,
                    convert_to_numpy=True,
                    batch_size=len(processed_texts)
                )
            
            # Dimenzió ellenőrzés és korrekció
            if embeddings.shape[1] != self.dimension:
                if embeddings.shape[1] > self.dimension:
                    embeddings = embeddings[:, :self.dimension]
                else:
                    padding = np.zeros((embeddings.shape[0], self.dimension - embeddings.shape[1]))
                    embeddings = np.hstack([embeddings, padding])
            
            # Performance tracking
            batch_time = time.time() - batch_start_time
            self.batch_times.append(batch_time)
            self.processed_count += len(batch_texts)
            
            return embeddings.astype(np.float32)
            
        except Exception as e:
            logger.error(f"Batch feldolgozási hiba: {e}")
            self.failed_count += len(batch_texts)
            # Fallback: NaN vektorok
            return np.full((len(batch_texts), self.dimension), np.nan, dtype=np.float32)
        
        finally:
            # Memória cleanup minden batch után
            if self.processed_count % 100 == 0:
                smart_memory_cleanup()
    
    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """Optimalizált embedding generálás."""
        total_texts = len(texts)
        logger.info(f"Optimalizált embedding generálás {total_texts:,} szöveghez")
        
        if not texts:
            return np.array([])
        
        # Text length analysis for adaptive batching
        text_lengths = [len(text) for text in texts]
        dynamic_batch_size = adaptive_batch_size(text_lengths, BATCH_SIZE)
        
        logger.info(f"Adaptív batch méret: {dynamic_batch_size}")
        logger.info(f"Átlagos szöveghossz: {np.mean(text_lengths):.0f} karakter")
        
        all_embeddings = []
        total_batches = (total_texts + dynamic_batch_size - 1) // dynamic_batch_size
        
        # Enhanced progress bar
        with tqdm(
            total=total_batches, 
            desc="Embedding generálás", 
            unit="batch",
            bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}]'
        ) as pbar:
            
            for i in range(0, total_texts, dynamic_batch_size):
                batch_texts = texts[i:i + dynamic_batch_size]
                
                # Memory monitoring
                memory_info = self._monitor_memory()
                
                # Batch processing
                batch_embeddings = self._process_batch_with_optimization(batch_texts)
                all_embeddings.extend(batch_embeddings.tolist())
                
                # Progress update with performance metrics
                if self.batch_times:
                    avg_batch_time = np.mean(self.batch_times[-10:])  # Last 10 batches
                    texts_per_second = dynamic_batch_size / avg_batch_time
                    
                    pbar.set_postfix({
                        'TPS': f'{texts_per_second:.1f}',
                        'Mem': f'{memory_info.get("allocated_gb", 0):.1f}GB',
                        'Fail': self.failed_count
                    })
                
                pbar.update(1)
                
                # Adaptive memory cleanup
                if memory_info.get('allocated_gb', 0) > MEMORY_LIMIT_GB * 0.8:
                    smart_memory_cleanup()
        
        # Final statistics
        total_time = sum(self.batch_times)
        self.total_processing_time = total_time
        
        logger.info(f"Optimalizált generálás befejezve!")
        logger.info(f"Összesített statisztikák:")
        logger.info(f"   - Feldolgozott: {self.processed_count:,}/{total_texts:,}")
        logger.info(f"   - Sikertelen: {self.failed_count:,}")
        logger.info(f"   - Teljes idő: {total_time/3600:.2f} óra")
        logger.info(f"   - Átlag sebesség: {self.processed_count/total_time:.1f} szöveg/sec")
        logger.info(f"   - Csúcs memória: {self.peak_memory_usage:.1f}GB")
        
        return np.array(all_embeddings, dtype=np.float32)

# Embedding generátor inicializálása
logger.info("Optimalizált Qwen3-8B generátor létrehozása...")
embedding_generator = OptimizedQwen3EmbeddingGenerator()


In [ ]:
#@title Főfolyamat - Optimalizált Chunk-alapú Feldolgozás
def process_embeddings_optimized():
    """
    Optimalizált embedding feldolgozás nagy adathalmazokhoz.
    
    Főbb optimalizációk:
    - Chunk-alapú feldolgozás
    - Streaming Parquet írás
    - Memory-mapped file handling
    - Adaptív batch sizing
    - Real-time performance monitoring
    """
    
    start_time = time.time()
    logger.info("Optimalizált embedding feldolgozás kezdése...")
    
    # 1. Adatok betöltése és elemzése
    logger.info(f"CSV betöltése: {INPUT_CSV}")
    
    try:
        # Első betöltés a méret és struktúra ellenőrzéséhez
        df_sample = pd.read_csv(INPUT_CSV, nrows=1000)
        logger.info(f"Sample betöltve: {len(df_sample)} sor")
        
        # Teljes fájl méret becslése
        total_rows = sum(1 for _ in open(INPUT_CSV, 'r', encoding='utf-8')) - 1  # -1 header
        logger.info(f"Becsült teljes sorok: {total_rows:,}")
        
        # Oszlop ellenőrzés
        required_columns = ['text', 'doc_id']
        missing_columns = [col for col in required_columns if col not in df_sample.columns]
        if missing_columns:
            raise ValueError(f"Hiányzó oszlopok: {missing_columns}")
        
        logger.info(f"Oszlop validáció sikeres")
        
    except Exception as e:
        logger.error(f"Adatbetöltési hiba: {e}")
        raise
    
    # 2. Chunk-alapú feldolgozás optimalizálással
    all_results = []
    processed_rows = 0
    chunk_number = 0
    
    # Performance tracking
    chunk_times = []
    memory_usage_log = []
    
    logger.info(f"Chunk-alapú feldolgozás kezdése (chunk méret: {CHUNKSIZE:,})")
    
    # Enhanced progress tracking
    total_chunks = (total_rows + CHUNKSIZE - 1) // CHUNKSIZE
    
    with tqdm(total=total_chunks, desc="Chunk Processing", unit="chunk") as chunk_pbar:
        
        for chunk_df in pd.read_csv(INPUT_CSV, chunksize=CHUNKSIZE, encoding='utf-8'):
            chunk_start_time = time.time()
            chunk_number += 1
            
            logger.info(f"Chunk {chunk_number}/{total_chunks} feldolgozása ({len(chunk_df):,} sor)")
            
            try:
                # Szöveg validáció és tisztítás
                chunk_df = chunk_df.dropna(subset=['text', 'doc_id'])
                chunk_df['text'] = chunk_df['text'].astype(str)
                
                valid_rows = len(chunk_df)
                if valid_rows == 0:
                    logger.warning(f"Chunk {chunk_number}: nincs érvényes adat")
                    continue
                
                texts = chunk_df['text'].tolist()
                
                # Optimalizált embedding generálás
                embeddings = embedding_generator.generate_embeddings(texts)
                
                # Eredmények hozzáadása a DataFrame-hez
                chunk_df['embedding'] = embeddings.tolist()
                
                # Metadata JSON generálás optimalizáltan
                chunk_df['metadata_json'] = chunk_df.apply(create_metadata_json, axis=1)
                
                # Memória optimalizáció: csak szükséges oszlopok
                output_columns = ['doc_id', 'birosag', 'jogterulet', 'hatarozat_id_mappa', 
                                'text', 'embedding', 'metadata_json']
                available_columns = [col for col in output_columns if col in chunk_df.columns]
                chunk_df_output = chunk_df[available_columns].copy()
                
                all_results.append(chunk_df_output)
                processed_rows += valid_rows
                
                # Performance metrics
                chunk_time = time.time() - chunk_start_time
                chunk_times.append(chunk_time)
                
                # Memory monitoring
                current_memory = psutil.virtual_memory()
                memory_usage_log.append({
                    'chunk': chunk_number,
                    'memory_used_gb': (current_memory.total - current_memory.available) / (1024**3),
                    'memory_percent': current_memory.percent
                })
                
                # Progress update with metrics
                avg_chunk_time = np.mean(chunk_times[-5:])  # Last 5 chunks
                rows_per_second = valid_rows / chunk_time
                
                chunk_pbar.set_postfix({
                    'Rows/s': f'{rows_per_second:.1f}',
                    'AvgTime': f'{avg_chunk_time:.1f}s',
                    'Memory': f'{memory_usage_log[-1]["memory_percent"]:.1f}%',
                    'Processed': f'{processed_rows:,}'
                })
                
                chunk_pbar.update(1)
                
                # Memória kezelés nagy chunk-oknál
                if chunk_number % 5 == 0:
                    smart_memory_cleanup()
                    logger.info(f"Memória tisztítás chunk {chunk_number} után")
                
                # ETA becslés
                if len(chunk_times) >= 3:
                    remaining_chunks = total_chunks - chunk_number
                    estimated_remaining_time = remaining_chunks * avg_chunk_time
                    logger.info(f"Becsült hátralévő idő: {estimated_remaining_time/3600:.2f} óra")
                
            except Exception as e:
                logger.error(f"Chunk {chunk_number} feldolgozási hiba: {e}")
                continue
    
    # 3. Eredmények kombinálása és mentése
    logger.info("Chunk-ok kombinálása...")
    
    if not all_results:
        raise ValueError("Nincs feldolgozott adat a mentéshez!")
    
    try:
        # Kombinálás memóriahatékony módon
        final_df = pd.concat(all_results, ignore_index=True)
        logger.info(f"Kombinált DataFrame: {len(final_df):,} sor")
        
        # Utolsó validáció
        embedding_check = final_df['embedding'].apply(lambda x: len(x) == 8192 if isinstance(x, list) else False)
        valid_embeddings = embedding_check.sum()
        
        logger.info(f"Érvényes embeddingek: {valid_embeddings:,}/{len(final_df):,}")
        
        # Parquet mentés optimalizáltan
        logger.info(f"Parquet mentés: {OUTPUT_PARQUET}")
        
        final_df.to_parquet(
            OUTPUT_PARQUET,
            engine='pyarrow',
            index=False,
            compression='snappy'
        )
        
        logger.info(f"Parquet fájl sikeresen mentve")
        
    except Exception as e:
        logger.error(f"Mentési hiba: {e}")
        raise
    
    # 4. Végső statisztikák
    total_time = time.time() - start_time
    
    logger.info("OPTIMALIZÁLT FELDOLGOZÁS BEFEJEZVE!")
    logger.info("=" * 60)
    logger.info(f"Végső statisztikák:")
    logger.info(f"   Feldolgozott sorok: {processed_rows:,}")
    logger.info(f"   Feldolgozott chunk-ok: {chunk_number}")
    logger.info(f"   Teljes futásidő: {total_time/3600:.2f} óra")
    logger.info(f"   Átlag feldolgozási sebesség: {processed_rows/total_time:.1f} sor/sec")
    logger.info(f"   Kimeneti fájl: {OUTPUT_PARQUET}")
    logger.info(f"   Embedding generátor statisztikák:")
    logger.info(f"      - Sikeres: {embedding_generator.processed_count:,}")
    logger.info(f"      - Sikertelen: {embedding_generator.failed_count:,}")
    logger.info(f"      - Csúcs memória: {embedding_generator.peak_memory_usage:.1f}GB")
    logger.info("=" * 60)
    
    return {
        'processed_rows': processed_rows,
        'total_time_hours': total_time / 3600,
        'average_speed': processed_rows / total_time,
        'output_file': OUTPUT_PARQUET,
        'chunk_count': chunk_number,
        'embedding_stats': {
            'successful': embedding_generator.processed_count,
            'failed': embedding_generator.failed_count,
            'peak_memory_gb': embedding_generator.peak_memory_usage
        }
    }

# Főfolyamat indítása
logger.info("Optimalizált főfolyamat indítása...")
results = process_embeddings_optimized()


In [ ]:
#@title Eredmények Validálása és Végső Statisztikák
def validate_and_finalize_results():
    """
    Végső eredmények validálása és részletes statisztikák.
    """
    
    logger.info("Eredmények validálása és végső statisztikák...")
    
    try:
        # Parquet fájl betöltése validáláshoz
        logger.info(f"Validációs betöltés: {OUTPUT_PARQUET}")
        validation_df = pd.read_parquet(OUTPUT_PARQUET)
        
        # Alapvető statisztikák
        total_rows = len(validation_df)
        logger.info(f"Betöltött sorok: {total_rows:,}")
        
        # Embedding validáció
        embedding_stats = {
            'total_embeddings': total_rows,
            'non_null_embeddings': validation_df['embedding'].notna().sum(),
            'correct_dimension': 0,
            'nan_embeddings': 0,
            'zero_embeddings': 0
        }
        
        logger.info("Embedding validáció...")
        
        for idx, emb in tqdm(enumerate(validation_df['embedding']), 
                           total=len(validation_df), 
                           desc="Validálás"):
            if emb is not None and isinstance(emb, list):
                if len(emb) == 8192:
                    embedding_stats['correct_dimension'] += 1
                    
                    # NaN és zero ellenőrzés
                    emb_array = np.array(emb)
                    if np.any(np.isnan(emb_array)):
                        embedding_stats['nan_embeddings'] += 1
                    elif np.all(emb_array == 0):
                        embedding_stats['zero_embeddings'] += 1
        
        # Metadata validáció
        metadata_stats = {
            'total_metadata': validation_df['metadata_json'].notna().sum(),
            'valid_json': 0,
            'doc_ids_present': validation_df['doc_id'].notna().sum(),
            'unique_doc_ids': validation_df['doc_id'].nunique()
        }
        
        logger.info("Metadata validáció...")
        
        for metadata_json in validation_df['metadata_json'].dropna():
            try:
                json.loads(metadata_json)
                metadata_stats['valid_json'] += 1
            except:
                pass
        
        # Jogterület és bíróság statisztikák
        categorical_stats = {
            'unique_birosagok': validation_df['birosag'].nunique() if 'birosag' in validation_df.columns else 0,
            'unique_jogteruletek': validation_df['jogterulet'].nunique() if 'jogterulet' in validation_df.columns else 0
        }
        
        if 'birosag' in validation_df.columns:
            top_birosagok = validation_df['birosag'].value_counts().head(5)
        
        if 'jogterulet' in validation_df.columns:
            top_jogteruletek = validation_df['jogterulet'].value_counts().head(5)
        
        # Eredmények megjelenítése
        logger.info("\nVÉGSŐ VALIDÁCIÓS EREDMÉNYEK")
        logger.info("=" * 80)
        
        logger.info("ÁLTALÁNOS STATISZTIKÁK:")
        logger.info(f"   Összes sor: {total_rows:,}")
        logger.info(f"   Egyedi doc_id-k: {metadata_stats['unique_doc_ids']:,}")
        logger.info(f"   Egyedi bíróságok: {categorical_stats['unique_birosagok']:,}")
        logger.info(f"   Egyedi jogterületek: {categorical_stats['unique_jogteruletek']:,}")
        
        logger.info("\nEMBEDDING MINŐSÉG:")
        logger.info(f"   Helyes dimenzió (8192): {embedding_stats['correct_dimension']:,}/{total_rows:,} "
                   f"({embedding_stats['correct_dimension']/total_rows*100:.2f}%)")
        logger.info(f"   NaN vektorok: {embedding_stats['nan_embeddings']:,}")
        logger.info(f"   Nulla vektorok: {embedding_stats['zero_embeddings']:,}")
        
        quality_score = (embedding_stats['correct_dimension'] - embedding_stats['nan_embeddings'] - 
                        embedding_stats['zero_embeddings']) / total_rows * 100
        logger.info(f"   Minőségi pontszám: {quality_score:.2f}%")
        
        logger.info("\nMETADATA MINŐSÉG:")
        logger.info(f"   Érvényes JSON: {metadata_stats['valid_json']:,}/{metadata_stats['total_metadata']:,} "
                   f"({metadata_stats['valid_json']/metadata_stats['total_metadata']*100:.2f}%)")
        
        if 'birosag' in validation_df.columns:
            logger.info("\nTOP 5 BÍRÓSÁG:")
            for birosag, count in top_birosagok.items():
                logger.info(f"   • {birosag}: {count:,} dokumentum")
        
        if 'jogterulet' in validation_df.columns:
            logger.info("\nTOP 5 JOGTERÜLET:")
            for jogterulet, count in top_jogteruletek.items():
                logger.info(f"   • {jogterulet}: {count:,} dokumentum")
        
        logger.info("\nTELJESÍTMÉNY ÖSSZEFOGLALÓ:")
        if 'results' in globals():
            logger.info(f"   Teljes futásidő: {results['total_time_hours']:.2f} óra")
            logger.info(f"   Átlag sebesség: {results['average_speed']:.1f} sor/sec")
            logger.info(f"   Feldolgozott chunk-ok: {results['chunk_count']}")
            logger.info(f"   Csúcs memória: {results['embedding_stats']['peak_memory_gb']:.1f}GB")
        
        logger.info("\nKOMPATIBILITÁS ELLENŐRZÉS:")
        faiss_compatible = embedding_stats['correct_dimension'] == total_rows
        graph_compatible = 'doc_id' in validation_df.columns and metadata_stats['doc_ids_present'] > 0
        
        logger.info(f"   FAISS index kompatibilis: {'IGEN' if faiss_compatible else 'NEM'}")
        logger.info(f"   Gráf builder kompatibilis: {'IGEN' if graph_compatible else 'NEM'}")
        logger.info(f"   Downstream pipeline ready: {'IGEN' if faiss_compatible and graph_compatible else 'NEM'}")
        
        logger.info("=" * 80)
        
        # Fájl méret információ
        file_size = os.path.getsize(OUTPUT_PARQUET) / (1024**3)
        logger.info(f"Kimeneti fájl mérete: {file_size:.2f} GB")
        
        return {
            'validation_successful': True,
            'total_rows': total_rows,
            'embedding_stats': embedding_stats,
            'metadata_stats': metadata_stats,
            'categorical_stats': categorical_stats,
            'quality_score': quality_score,
            'faiss_compatible': faiss_compatible,
            'graph_compatible': graph_compatible,
            'file_size_gb': file_size
        }
        
    except Exception as e:
        logger.error(f"Validációs hiba: {e}")
        return {'validation_successful': False, 'error': str(e)}

# Validáció futtatása
logger.info("Végső validáció indítása...")
validation_results = validate_and_finalize_results()

if validation_results['validation_successful']:
    print("\nOPTIMALIZÁLT QWEN3-8B EMBEDDING GENERÁLÁS SIKERESEN BEFEJEZVE!")
    print(f"Minőségi pontszám: {validation_results['quality_score']:.2f}%")
    print(f"Fájl méret: {validation_results['file_size_gb']:.2f} GB")
    print(f"FAISS kompatibilis: {'IGEN' if validation_results['faiss_compatible'] else 'NEM'}")
    print(f"Gráf kompatibilis: {'IGEN' if validation_results['graph_compatible'] else 'NEM'}")
else:
    print(f"Validáció sikertelen: {validation_results.get('error', 'Ismeretlen hiba')}")


In [ ]:

#@title Eredmények Validálása és Végső Statisztikák
def validate_and_finalize_results():
    """
    Végső eredmények validálása és részletes statisztikák.
    """
    
    logger.info("Eredmények validálása és végső statisztikák...")
    
    try:
        # Parquet fájl betöltése validáláshoz
        logger.info(f"Validációs betöltés: {OUTPUT_PARQUET}")
        validation_df = pd.read_parquet(OUTPUT_PARQUET)
        
        # Alapvető statisztikák
        total_rows = len(validation_df)
        logger.info(f"Betöltött sorok: {total_rows:,}")
        
        # Embedding validáció
        embedding_stats = {
            'total_embeddings': total_rows,
            'non_null_embeddings': validation_df['embedding'].notna().sum(),
            'correct_dimension': 0,
            'nan_embeddings': 0,
            'zero_embeddings': 0
        }
        
        logger.info("Embedding validáció...")
        
        for idx, emb in tqdm(enumerate(validation_df['embedding']), 
                           total=len(validation_df), 
                           desc="Validálás"):
            if emb is not None and isinstance(emb, list):
                if len(emb) == 8192:
                    embedding_stats['correct_dimension'] += 1
                    
                    # NaN és zero ellenőrzés
                    emb_array = np.array(emb)
                    if np.any(np.isnan(emb_array)):
                        embedding_stats['nan_embeddings'] += 1
                    elif np.all(emb_array == 0):
                        embedding_stats['zero_embeddings'] += 1
        
        # Metadata validáció
        metadata_stats = {
            'total_metadata': validation_df['metadata_json'].notna().sum(),
            'valid_json': 0,
            'doc_ids_present': validation_df['doc_id'].notna().sum(),
            'unique_doc_ids': validation_df['doc_id'].nunique()
        }
        
        logger.info("Metadata validáció...")
        
        for metadata_json in validation_df['metadata_json'].dropna():
            try:
                json.loads(metadata_json)
                metadata_stats['valid_json'] += 1
            except:
                pass
        
        # Jogterület és bíróság statisztikák
        categorical_stats = {
            'unique_birosagok': validation_df['birosag'].nunique() if 'birosag' in validation_df.columns else 0,
            'unique_jogteruletek': validation_df['jogterulet'].nunique() if 'jogterulet' in validation_df.columns else 0
        }
        
        if 'birosag' in validation_df.columns:
            top_birosagok = validation_df['birosag'].value_counts().head(5)
        
        if 'jogterulet' in validation_df.columns:
            top_jogteruletek = validation_df['jogterulet'].value_counts().head(5)
        
        # Eredmények megjelenítése
        logger.info("\nVÉGSŐ VALIDÁCIÓS EREDMÉNYEK")
        logger.info("=" * 80)
        
        logger.info("ÁLTALÁNOS STATISZTIKÁK:")
        logger.info(f"   Összes sor: {total_rows:,}")
        logger.info(f"   Egyedi doc_id-k: {metadata_stats['unique_doc_ids']:,}")
        logger.info(f"   Egyedi bíróságok: {categorical_stats['unique_birosagok']:,}")
        logger.info(f"   Egyedi jogterületek: {categorical_stats['unique_jogteruletek']:,}")
        
        logger.info("\nEMBEDDING MINŐSÉG:")
        logger.info(f"   Helyes dimenzió (8192): {embedding_stats['correct_dimension']:,}/{total_rows:,} "
                   f"({embedding_stats['correct_dimension']/total_rows*100:.2f}%)")
        logger.info(f"   NaN vektorok: {embedding_stats['nan_embeddings']:,}")
        logger.info(f"   Nulla vektorok: {embedding_stats['zero_embeddings']:,}")
        
        quality_score = (embedding_stats['correct_dimension'] - embedding_stats['nan_embeddings'] - 
                        embedding_stats['zero_embeddings']) / total_rows * 100
        logger.info(f"   Minőségi pontszám: {quality_score:.2f}%")
        
        logger.info("\nMETADATA MINŐSÉG:")
        logger.info(f"   Érvényes JSON: {metadata_stats['valid_json']:,}/{metadata_stats['total_metadata']:,} "
                   f"({metadata_stats['valid_json']/metadata_stats['total_metadata']*100:.2f}%)")
        
        if 'birosag' in validation_df.columns:
            logger.info("\nTOP 5 BÍRÓSÁG:")
            for birosag, count in top_birosagok.items():
                logger.info(f"   • {birosag}: {count:,} dokumentum")
        
        if 'jogterulet' in validation_df.columns:
            logger.info("\nTOP 5 JOGTERÜLET:")
            for jogterulet, count in top_jogteruletek.items():
                logger.info(f"   • {jogterulet}: {count:,} dokumentum")
        
        logger.info("\nTELJESÍTMÉNY ÖSSZEFOGLALÓ:")
        if 'results' in globals():
            logger.info(f"   Teljes futásidő: {results['total_time_hours']:.2f} óra")
            logger.info(f"   Átlag sebesség: {results['average_speed']:.1f} sor/sec")
            logger.info(f"   Feldolgozott chunk-ok: {results['chunk_count']}")
            logger.info(f"   Csúcs memória: {results['embedding_stats']['peak_memory_gb']:.1f}GB")
        
        logger.info("\nKOMPATIBILITÁS ELLENŐRZÉS:")
        faiss_compatible = embedding_stats['correct_dimension'] == total_rows
        graph_compatible = 'doc_id' in validation_df.columns and metadata_stats['doc_ids_present'] > 0
        
        logger.info(f"   FAISS index kompatibilis: {'IGEN' if faiss_compatible else 'NEM'}")
        logger.info(f"   Gráf builder kompatibilis: {'IGEN' if graph_compatible else 'NEM'}")
        logger.info(f"   Downstream pipeline ready: {'IGEN' if faiss_compatible and graph_compatible else 'NEM'}")
        
        logger.info("=" * 80)
        
        # Fájl méret információ
        file_size = os.path.getsize(OUTPUT_PARQUET) / (1024**3)
        logger.info(f"Kimeneti fájl mérete: {file_size:.2f} GB")
        
        return {
            'validation_successful': True,
            'total_rows': total_rows,
            'embedding_stats': embedding_stats,
            'metadata_stats': metadata_stats,
            'categorical_stats': categorical_stats,
            'quality_score': quality_score,
            'faiss_compatible': faiss_compatible,
            'graph_compatible': graph_compatible,
            'file_size_gb': file_size
        }
        
    except Exception as e:
        logger.error(f"Validációs hiba: {e}")
        return {'validation_successful': False, 'error': str(e)}

# Validáció futtatása
logger.info("Végső validáció indítása...")
validation_results = validate_and_finalize_results()

if validation_results['validation_successful']:
    print("\nOPTIMALIZÁLT QWEN3-8B EMBEDDING GENERÁLÁS SIKERESEN BEFEJEZVE!")
    print(f"Minőségi pontszám: {validation_results['quality_score']:.2f}%")
    print(f"Fájl méret: {validation_results['file_size_gb']:.2f} GB")
    print(f"FAISS kompatibilis: {'IGEN' if validation_results['faiss_compatible'] else 'NEM'}")
    print(f"Gráf kompatibilis: {'IGEN' if validation_results['graph_compatible'] else 'NEM'}")
else:
    print(f"Validáció sikertelen: {validation_results.get('error', 'Ismeretlen hiba')}")